<a href="https://colab.research.google.com/github/jessicayeh35/Expense-project/blob/master/full_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jazz Hands for Autism + DataRes: Scheduling Optimization Project

The below code file builds out student and instructor schedules based on 2025 availability data provided by JHFA students & instructors in a Google Form.

The current version of the model, using data from 2025 Q1 is able to match all classes for students based on their required classes, primary instruments, availability and so on. The model is able to match most instructors with scheduled classes as well, although we find that some instructors are assigned less classes by the model, which can be fixed by manually reassigning classes from some instructors to others to balance their schedule.

Please follow the steps given below to run your own model with future data and generate a fresh set of schedules. We have included a 2025 sample for reference.

Happy scheduling!

### Step 1: Importing Data

- Run the first cell below to download `ortools`, the required scheduling optimization library for building the model
- Run the second cell and sign in to Google Drive with any of the email IDs associated with the 'JHFA' data folder shared with you
- Run the third cell to import data: feel free to customize the file path before `/JHFA/optimization_with_data/student_df_simplified.pkl` as per your own Google Drive

In [ ]:
%pip install ortools

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

student_df = pd.read_pickle('/content/drive/My Drive/JHFA/cleaned_data/student_df_simplified.pkl')
instructor_df = pd.read_pickle('/content/drive/My Drive/JHFA/cleaned_data/instructor_df_simplified.pkl')

# Un-comment the below statements to get a preview of the data files for Students and Instructors
#print(student_df.head(1))
#print(instructor_df.head(1))

### Step 2: Hard coding variables

- We hard coded some variables such as mandatory classes & group classes.
- Group class limit: number of students allowed in each group class
- Room preferences: based on the room preference excel file & scheduling documents provided

Changing these will affect the rest of the model outputs, but please do so if there is a change in the class structure.

In [ ]:
mandatory_classes = ["MUSIC THEORY", "GUIDANCE COUNSELING", "CAREER/JOB COACHING"]
group_classes = ["ENSEMBLE (BAND)", "CHOIR", "MUSICIANSHIP"]
GROUP_CLASS_LIMIT = 5
room_preferences = {
    'drums': ['101B'],
    '1:1': ['206A'],
    'piano': ['206B', '206D'],
    'voi': ['206D'],
    'vocal': ['206D'],
    'guitar': ['206D'],
    # 'online': ['101A', '206C'],
    'ensemble': ['206D', '101B'],
    'group': ['101A', '206A'],
    'theory': ['206B'],
    'music tech': ['206C'],
    'dj': ['206C'],
    'year2': ['206B'],
    'elective': ['206B', '206C']
}

# Room list and mappings
room_list = ['101A', '101B', '206A', '206B', '206C', '206D']
room_to_index = {room: idx for idx, room in enumerate(room_list)}
index_to_room = {idx: room for room, idx in room_to_index.items()}

### Step 3: Final Model

- Run the below cell to initiate the Scheduling Model. It should take about a minute to run.
- The model will also print out its output. Cells in 'Step 4' will convert this output to a csv file which you can download to your system.

In [ ]:
from collections import defaultdict
from ortools.sat.python import cp_model

mandatory_classes = ["MUSIC THEORY", "GUIDANCE COUNSELING", "CAREER/JOB COACHING"]
group_classes = ["ENSEMBLE (BAND)", "CHOIR", "MUSICIANSHIP"]
GROUP_CLASS_LIMIT = 5
room_preferences = {
    'drums': ['101B'],
    '1:1': ['206A'],
    'piano': ['206B', '206D'],
    'voi': ['206D'],
    'vocal': ['206D'],
    'guitar': ['206D'],
    # 'online': ['101A', '206C'],
    'ensemble': ['206D', '101B'],
    'group': ['101A', '206A'],
    'theory': ['206B'],
    'music tech': ['206C'],
    'dj': ['206C'],
    'year2': ['206B'],
    'elective': ['206B', '206C']
}

# Room list and mappings
room_list = ['101A', '101B', '206A', '206B', '206C', '206D']
room_to_index = {room: idx for idx, room in enumerate(room_list)}
index_to_room = {idx: room for room, idx in room_to_index.items()}

model = cp_model.CpModel()

assignments = {}
assignment_weights = {}
room_vars = {}

primary_instrument_penalty = 5
spread_penalty_weight = 2
block_penalty_weight = 3
room_penalty_weight = 2

spread_penalties = []

# === Create assignment variables and room assignment variables ===
for student_idx, student_row in student_df.iterrows():
    student_name = student_row['Student Name']
    student_classes = student_row['Class List']
    student_avail = student_row['Availability']
    primary_instrument = student_row['Primary Instrument']

    for student_class in student_classes:
        is_mandatory = student_class in mandatory_classes
        student_class_lower = student_class.lower()

        for instructor_idx, instructor_row in instructor_df.iterrows():
            instructor_name = instructor_row['Instructor Name']
            if student_class not in instructor_row['Class List']:
                continue

            for day in student_avail:
                if day in instructor_row['Availability']:
                    common_times = set(student_avail[day]).intersection(instructor_row['Availability'][day])
                    for time in common_times:
                        var_name = f'{student_name}-{student_class}-{instructor_name}-{day}-{time}'
                        assignments[var_name] = model.NewBoolVar(var_name)
                        assignment_weights[var_name] = 10 if is_mandatory else 1

                        allowed_rooms = []

                        instr_mode = str(
                            instructor_row.get('Availability Mode', {}).get(day, '')
                            if isinstance(instructor_row.get('Availability Mode', {}), dict)
                            else instructor_row.get('Availability Mode', '')
                        ).lower()

                        stud_mode  = str(
                            student_row.get('Availability Mode', {}).get(day, '')
                            if isinstance(student_row.get('Availability Mode', {}), dict)
                            else student_row.get('Availability Mode', '')
                        ).lower()

                        modality = 'online' if (
                            'online' in student_class_lower or
                            instr_mode in ('online', 'zoom', 'remote') or
                            stud_mode  in ('online', 'zoom', 'remote')
                        ) else 'inperson'


                        if modality == 'online':
                            room_vars[var_name] = None
                        else:
                            # assign allowed_rooms based on tags like piano, vocal, etc.
                            if 'drums' in student_class_lower:
                                allowed_rooms = [room_to_index['101B']]
                            elif '1:1' in student_class_lower:
                                allowed_rooms = [room_to_index['206A']]
                            elif 'piano' in student_class_lower:
                                allowed_rooms = [room_to_index[r] for r in ['206B', '206D']]
                            elif any(tag in student_class_lower for tag in ['voi', 'vocal', 'guitar']):
                                allowed_rooms = [room_to_index['206D']]
                            elif any(tag in student_class_lower for tag in ['ensemble', 'orchestra', 'choir', 'jazz band', 'group']):
                                allowed_rooms = [room_to_index[r] for r in ['101A', '206A', '206D']]
                                if 'drums' in student_class_lower:
                                    allowed_rooms.append(room_to_index['101B'])
                            elif any(tag in student_class_lower for tag in ['theory', 'year2', 'elective']):
                                allowed_rooms = [room_to_index[r] for r in ['206B', '206C']]
                            elif any(tag in student_class_lower for tag in ['music tech', 'dj']):
                                allowed_rooms = [room_to_index['206C']]
                            else:
                                allowed_rooms = list(room_to_index.values())

                            room_var = model.NewIntVarFromDomain(cp_model.Domain.FromValues(allowed_rooms), f"room_{var_name}")
                            room_vars[var_name] = room_var

                            preferred_rooms = set(room_to_index[r] for r in room_preferences.get(student_class_lower, []))
                            if preferred_rooms:
                                not_preferred = model.NewBoolVar(f"not_preferred_{var_name}")
                                model.AddAllowedAssignments([room_var], [[r] for r in preferred_rooms]).OnlyEnforceIf(not_preferred.Not())
                                model.AddForbiddenAssignments([room_var], [[r] for r in range(len(room_list)) if r not in preferred_rooms]).OnlyEnforceIf(not_preferred)
                                spread_penalties.append(room_penalty_weight * not_preferred)


# === One assignment per class per student ===
for student_idx, student_row in student_df.iterrows():
    student_name = student_row['Student Name']
    for student_class in student_row['Class List']:
        class_assignments = [assignments[var] for var in assignments if var.startswith(f'{student_name}-{student_class}-')]
        if class_assignments:
            model.Add(sum(class_assignments) == 1)

# === Prevent overlapping classes per student ===
for student_idx, student_row in student_df.iterrows():
    student_name = student_row['Student Name']
    student_avail = student_row['Availability']

    for day, hours in student_avail.items():
        for hour in hours:
            overlapping_vars = [assignments[var] for var in assignments if var.startswith(f"{student_name}-") and f"-{day}-{hour}" in var]
            if overlapping_vars:
                model.Add(sum(overlapping_vars) <= 1)

# === Prevent overlapping classes per instructor, enforce group class limits ===
for instructor_idx, instructor_row in instructor_df.iterrows():
    instructor_name = instructor_row['Instructor Name']
    instructor_avail = instructor_row['Availability']

    for day, hours in instructor_avail.items():
        for hour in hours:
            overlapping_vars = [(var, var.split('-')[1]) for var in assignments if f"-{instructor_name}-" in var and f"-{day}-{hour}" in var]

            group_vars = [assignments[var] for var, cls in overlapping_vars if cls in group_classes]
            indiv_vars = [assignments[var] for var, cls in overlapping_vars if cls not in group_classes]

            if group_vars:
                model.Add(sum(group_vars) <= GROUP_CLASS_LIMIT)
            if indiv_vars:
                model.Add(sum(indiv_vars) <= 1)

# === No double booking of rooms unless both classes are group classes ===
room_time_assignments = defaultdict(list)

for var_name, var in assignments.items():
    student_name, class_name, instructor_name, day, hour = var_name.rsplit('-', 4)
    class_name_norm = class_name.strip().upper()
    room_var = room_vars[var_name]

    room_time_assignments[(day, hour)].append((var_name, class_name_norm, room_var))

for (day, hour), assignments_at_time in room_time_assignments.items():
    n = len(assignments_at_time)
    for i in range(n):
        var1, cls1, r1 = assignments_at_time[i]
        for j in range(i + 1, n):
            var2, cls2, r2 = assignments_at_time[j]

            if cls1 in group_classes and cls2 in group_classes:
                continue

            b1 = assignments[var1]
            b2 = assignments[var2]

            both_assigned = model.NewBoolVar(f'both_assigned_{var1}_{var2}')
            model.AddBoolAnd([b1, b2]).OnlyEnforceIf(both_assigned)
            model.AddBoolOr([b1.Not(), b2.Not()]).OnlyEnforceIf(both_assigned.Not())

            if r1 is not None and r2 is not None:
                model.Add(r1 != r2).OnlyEnforceIf(both_assigned)

# === Encourage block schedule consistency ===
def get_block(hour):
    h = int(hour)
    if 9 <= h <= 12:
        return 'morning'
    elif 14 <= h <= 17:
        return 'afternoon'
    return None

for person_df, is_student in [(student_df, True), (instructor_df, False)]:
    for idx, row in person_df.iterrows():
        name = row['Student Name'] if is_student else row['Instructor Name']
        availability = row['Availability']
        modes = row.get('Availability Mode', {})

        for day in availability:
            block_assignments = {'morning': [], 'afternoon': []}
            block_modes = {'morning': set(), 'afternoon': set()}

            for hour in availability[day]:
                block = get_block(hour)
                if not block:
                    continue

                for var in assignments:
                    if f"-{day}-{hour}" in var and name in var:
                        block_assignments[block].append(assignments[var])
                        if not is_student:
                            mode = modes.get(day, '')
                            block_modes[block].add(mode)
                        else:
                            instr_name = var.split('-')[2]
                            instr_row = instructor_df[instructor_df['Instructor Name'] == instr_name]
                            if not instr_row.empty:
                                instr_mode = instr_row.iloc[0]['Availability Mode'].get(day, '')
                                block_modes[block].add(instr_mode)

            for block in ['morning', 'afternoon']:
                if block_assignments[block]:
                    total = len(block_assignments[block])
                    assigned_sum = model.NewIntVar(0, total, f'{name}_{day}_{block}_assigned')
                    model.Add(assigned_sum == sum(block_assignments[block]))

                    is_singleton = model.NewBoolVar(f'{name}_{day}_{block}_singleton')
                    model.Add(assigned_sum == 1).OnlyEnforceIf(is_singleton)
                    model.Add(assigned_sum != 1).OnlyEnforceIf(is_singleton.Not())
                    spread_penalties.append(block_penalty_weight * is_singleton)

                    if len(block_modes[block]) > 1:
                        spread_penalties.append(block_penalty_weight)

# === Objective function ===
model.Maximize(
    sum(assignment_weights[var] * assignments[var] for var in assignments) -
    spread_penalty_weight * sum(spread_penalties)
)

# === Solve ===
solver = cp_model.CpSolver()
status = solver.Solve(model)

# === Output results as in Model A ===
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    # --- Print Student Schedules ---
    for student_idx, student_row in student_df.iterrows():
        student_name = student_row['Student Name']
        student_classes = student_row['Class List']

        print(f"Schedule for {student_name}:")
        student_schedule = {day: [] for day in ['M', 'T', 'W', 'Th', 'F']}
        seen_assignments = set()
        matched_classes = set()
        primary_instrument_hours = 0

        for student_class in student_classes:
            for instructor_idx, instructor_row in instructor_df.iterrows():
                instructor_name = instructor_row['Instructor Name']
                for day in student_row['Availability']:
                    if day in instructor_row['Availability']:
                        common_times = set(student_row['Availability'][day]).intersection(instructor_row['Availability'][day])
                        for time in common_times:
                            var_name = f'{student_name}-{student_class}-{instructor_name}-{day}-{time}'
                            if var_name in assignments and solver.Value(assignments[var_name]) == 1:
                                if room_vars[var_name] is None:
                                    assigned_room = "Online"
                                else:
                                    assigned_room_idx = solver.Value(room_vars[var_name])
                                    assigned_room = index_to_room[assigned_room_idx]
                                display_str = f"{student_class} with {instructor_name} at {time} ({instructor_row['Availability Mode'].get(day, '')}), Room: {assigned_room}"
                                if (day, display_str) not in seen_assignments:
                                    student_schedule[day].append(display_str)
                                    seen_assignments.add((day, display_str))
                                    matched_classes.add(student_class)
                                    if student_class == student_row['Primary Instrument']:
                                        primary_instrument_hours += 1

        for day in ['M', 'T', 'W', 'Th', 'F']:
            if student_schedule[day]:
                print(f"  {day}:")
                for entry in sorted(student_schedule[day]):
                    print(f"    {entry}")
        print(f"  ✅ Matched {len(matched_classes)} out of {len(student_classes)} requested classes\n")
        #print(f"Primary instrument hours: {primary_instrument_hours}\n")

    # --- Print Instructor Schedules ---
    for instructor_idx, instructor_row in instructor_df.iterrows():
        instructor_name = instructor_row['Instructor Name']
        instructor_schedule = {day: [] for day in ['M', 'T', 'W', 'Th', 'F']}
        seen_instructor_assignments = set()

        # For each assignment involving this instructor
        for var_name in assignments:
            # Parse var_name format: student-class-instructor-day-time
            parts = var_name.split('-')
            if len(parts) < 5:
                continue
            student_name = parts[0]
            student_class = parts[1]
            instr_name = parts[2]
            day = parts[-2]
            time = parts[-1]

            if instr_name != instructor_name:
                continue
            if solver.Value(assignments[var_name]) == 1:
                if room_vars[var_name] is None:
                    assigned_room = "Online"
                else:
                    assigned_room_idx = solver.Value(room_vars[var_name])
                    assigned_room = index_to_room[assigned_room_idx]

                mode = instructor_row['Availability Mode'].get(day, '')
                display_str = f"{student_class} with {student_name} at {time} ({mode}), Room: {assigned_room}"
                if (day, display_str) not in seen_instructor_assignments:
                    instructor_schedule[day].append(display_str)
                    seen_instructor_assignments.add((day, display_str))

        print(f"Schedule for Instructor {instructor_name}:")
        for day in ['M', 'T', 'W', 'Th', 'F']:
            if instructor_schedule[day]:
                print(f"  {day}:")
                for entry in sorted(instructor_schedule[day]):
                    print(f"    {entry}")
        print()
else:
    print("No feasible schedule found.")


# === Create DataFrames ===
student_schedule_df = pd.DataFrame(student_schedule_df)
instructor_schedule_df = pd.DataFrame(instructor_schedule_df)

Schedule for Aaron :
  T:
    GUIDANCE COUNSELING with Em Smith at 12 (In-Person), Room: 101A
    MUSIC THEORY with Siv at 10 (Online), Room: Online
    PIANO with Siv at 11 (Online), Room: Online
  F:
    CAREER/JOB COACHING with Em Smith at 9 (In-Person), Room: 206B
    CONCERT REHEARSAL (Saturday Jam) with Rocco Matone at 11 (In-Person), Room: 101A
  ✅ Matched 5 out of 5 requested classes

Schedule for Brandi Pollard :
  M:
    CAREER/JOB COACHING with Em Smith at 10 (Online), Room: Online
    CHOIR with Evan O’Brien at 9 (Online), Room: Online
  Th:
    ENSEMBLE (BAND) with Evan O’Brien at 9 (Online), Room: Online
    MUSIC THEORY with Siv at 10 (Online), Room: Online
    VOICE with Rocco Matone at 11 (In-Person), Room: 206D
  F:
    CONCERT REHEARSAL (Saturday Jam) with Rocco Matone at 14 (In-Person), Room: 101B
    GUIDANCE COUNSELING with Em Smith at 16 (In-Person), Room: 101A
  ✅ Matched 7 out of 7 requested classes

Schedule for El Cid Catlin-Muñoz:
  Th:
    CONCERT REHEARSAL

### Step 4: Preview of Output Schedules (Pandas Dataframes)

- `student_schedule_df`: Provides the schedule for all students with matched classes, grouped by student name.
- `instructor_schedule_df`: Provides the schedule for all instructors with matched classes, grouped by instructor name.
- `master_calendar`: Provides a master calendar for all classes, ordered by times on Monday through Friday.

In [ ]:
#student schedules

student_schedule_df.head()


,Student Name,Class Name,Instructor Name,Day,Time,Room,Mode,Status
0,Aaron,CONCERT REHEARSAL (Saturday Jam),Rocco Matone,F,9,101A,In-Person,Assigned
1,Aaron,MUSIC THEORY,Siv,T,9,206B,Online,Assigned
2,Aaron,GUIDANCE COUNSELING,Em Smith,F,12,101A,In-Person,Assigned
3,Aaron,CAREER/JOB COACHING,Em Smith,F,10,101A,In-Person,Assigned
4,Aaron,PIANO,Siv,T,10,206B,Online,Assigned


In [ ]:
#instructor schedules

instructor_schedule_df.head()

,Instructor Name,Student Name,Class Name,Day,Time,Room,Mode
0,Rocco Matone,Aaron,CONCERT REHEARSAL (Saturday Jam),F,9,101A,In-Person
1,Rocco Matone,Brandi Pollard,CONCERT REHEARSAL (Saturday Jam),Th,14,101A,In-Person
2,Rocco Matone,Brandi Pollard,VOICE,Th,16,206D,In-Person
3,Rocco Matone,Brandon Wall,GUITAR,Th,10,206D,In-Person
4,Rocco Matone,Sam Williams,VOICE,Th,11,206D,In-Person


In [ ]:
# Master calendar for all classes

schedule_records = []

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    for var_name in assignments:
        if solver.Value(assignments[var_name]) == 1:
            student_name, student_class, instructor_name, day, time = var_name.rsplit('-', 4)
            room_var = room_vars.get(var_name)
            assigned_room = "Online" if room_var is None else index_to_room[solver.Value(room_var)]

            schedule_records.append({
                "Day": day,
                "Time": int(time),
                "TimeLabel": f"{int(time):02d}:00",
                "Class": student_class,
                "Student": student_name,
                "Instructor": instructor_name,
                "Room": assigned_room,
                "Entry": f"{student_class} - {student_name} / {instructor_name} ({assigned_room})"
            })

    # Create DataFrame and sort
    master_calendar = pd.DataFrame(schedule_records)
    day_order = {'M': 0, 'T': 1, 'W': 2, 'Th': 3, 'F': 4}
    master_calendar["DayIndex"] = master_calendar["Day"].map(day_order)
    master_calendar = master_calendar.sort_values(by=["DayIndex", "Time"]).drop(columns="DayIndex").reset_index(drop=True)

    # Return the final DataFrame
    master_calendar
else:
    print("No feasible schedule found.")


#printing out master calendar
master_calendar.head()

,Day,Time,TimeLabel,Class,Student,Instructor,Room,Entry
0,M,9,09:00,CHOIR,Brandi Pollard,Evan O’Brien,Online,CHOIR - Brandi Pollard / Evan O’Brien (Online)
1,M,9,09:00,MUSIC THEORY,Brandon Wall,Siv,206B,MUSIC THEORY - Brandon Wall / Siv (206B)
2,M,10,10:00,CAREER/JOB COACHING,Brandi Pollard,Em Smith,Online,CAREER/JOB COACHING - Brandi Pollard / Em Smi...
3,M,10,10:00,ENSEMBLE (BAND),Brandon Wall,Evan O’Brien,Online,ENSEMBLE (BAND) - Brandon Wall / Evan O’Brien...
4,M,11,11:00,ENSEMBLE (BAND),Laqota Gunn,Evan O’Brien,Online,ENSEMBLE (BAND) - Laqota Gunn / Evan O’Brien (...


### Step 5: Downloading the Dataframes to a PC

In [ ]:
#run this code chunk to save and download the student, instructor and master schedules

master_calendar.to_csv("calendar.csv", index=False)
student_schedule_df.to_csv("student_schedule.csv", index=False)
instructor_schedule_df.to_csv("instructor_schedule.csv", index=False)

from google.colab import files
files.download("calendar.csv")
files.download("student_schedule.csv")
files.download("instructor_schedule.csv")

# Archived Versions of the Model

**Model 1 - Original Strict Model**:
- This is a full model which includes all constraints, and will leave classes unmatched if all conditions are not met.
- It also provides locations to take online classes. This is something we improved upon in the Final Model (above).

In [ ]:
'''
# Model 1: Original strict model


from collections import defaultdict
from ortools.sat.python import cp_model

mandatory_classes = ["MUSIC THEORY", "GUIDANCE COUNSELING", "CAREER/JOB COACHING"]
group_classes = ["ENSEMBLE (BAND)", "CHOIR", "MUSICIANSHIP"]
GROUP_CLASS_LIMIT = 5
room_preferences = {
    'drums': ['101B'],
    '1:1': ['206A'],
    'piano': ['206B', '206D'],
    'voi': ['206D'],
    'vocal': ['206D'],
    'guitar': ['206D'],
    'online': ['101A', '206C'],
    'ensemble': ['206D', '101B'],
    'group': ['101A', '206A'],
    'theory': ['206B'],
    'music tech': ['206C'],
    'dj': ['206C'],
    'year2': ['206B'],
    'elective': ['206B', '206C']
}

# Room list and mappings
room_list = ['101A', '101B', '206A', '206B', '206C', '206D']
room_to_index = {room: idx for idx, room in enumerate(room_list)}
index_to_room = {idx: room for room, idx in room_to_index.items()}

model = cp_model.CpModel()

assignments = {}
assignment_weights = {}
room_vars = {}

primary_instrument_penalty = 5
spread_penalty_weight = 2
block_penalty_weight = 3
room_penalty_weight = 2

spread_penalties = []

# === Create assignment variables and room assignment variables ===
for student_idx, student_row in student_df.iterrows():
    student_name = student_row['Student Name']
    student_classes = student_row['Class List']
    student_avail = student_row['Availability']
    primary_instrument = student_row['Primary Instrument']

    for student_class in student_classes:
        is_mandatory = student_class in mandatory_classes
        student_class_lower = student_class.lower()

        for instructor_idx, instructor_row in instructor_df.iterrows():
            instructor_name = instructor_row['Instructor Name']
            if student_class not in instructor_row['Class List']:
                continue

            for day in student_avail:
                if day in instructor_row['Availability']:
                    common_times = set(student_avail[day]).intersection(instructor_row['Availability'][day])
                    for time in common_times:
                        var_name = f'{student_name}-{student_class}-{instructor_name}-{day}-{time}'
                        assignments[var_name] = model.NewBoolVar(var_name)
                        assignment_weights[var_name] = 10 if is_mandatory else 1

                        allowed_rooms = []
                        modality = 'online' if 'online' in student_class_lower else 'inperson'

                        if modality == 'online':
                            allowed_rooms = [room_to_index[r] for r in ['101A', '206C']]
                        else:
                            # instrument classes
                            if 'drums' in student_class_lower:
                                allowed_rooms = [room_to_index['101B']]
                            elif '1:1' in student_class_lower:
                                allowed_rooms = [room_to_index['206A']]
                            elif 'piano' in student_class_lower:
                                allowed_rooms = [room_to_index[r] for r in ['206B', '206D']]
                            elif any(tag in student_class_lower for tag in ['voi', 'vocal', 'guitar']):
                                allowed_rooms = [room_to_index['206D']]
                            elif any(tag in student_class_lower for tag in ['ensemble', 'orchestra', 'choir', 'jazz band', 'group']):
                                allowed_rooms = [room_to_index[r] for r in ['101A', '206A', '206D']]
                                if 'drums' in student_class_lower:
                                    allowed_rooms.append(room_to_index['101B'])
                            elif any(tag in student_class_lower for tag in ['theory', 'year2', 'elective']):
                                allowed_rooms = [room_to_index[r] for r in ['206B', '206C']]
                            elif any(tag in student_class_lower for tag in ['music tech', 'dj']):
                                allowed_rooms = [room_to_index['206C']]
                            else:
                                allowed_rooms = list(room_to_index.values())

                        room_var = model.NewIntVarFromDomain(cp_model.Domain.FromValues(allowed_rooms), f"room_{var_name}")
                        room_vars[var_name] = room_var

                        preferred_rooms = set(room_to_index[r] for r in room_preferences.get(student_class_lower, []))
                        if preferred_rooms:
                            not_preferred = model.NewBoolVar(f"not_preferred_{var_name}")
                            model.AddAllowedAssignments([room_var], [[r] for r in preferred_rooms]).OnlyEnforceIf(not_preferred.Not())
                            model.AddForbiddenAssignments([room_var], [[r] for r in range(len(room_list)) if r not in preferred_rooms]).OnlyEnforceIf(not_preferred)
                            spread_penalties.append(room_penalty_weight * not_preferred)

# === One assignment per class per student ===
for student_idx, student_row in student_df.iterrows():
    student_name = student_row['Student Name']
    for student_class in student_row['Class List']:
        class_assignments = [assignments[var] for var in assignments if var.startswith(f'{student_name}-{student_class}-')]
        if class_assignments:
            model.Add(sum(class_assignments) == 1)

# === Prevent overlapping classes per student ===
for student_idx, student_row in student_df.iterrows():
    student_name = student_row['Student Name']
    student_avail = student_row['Availability']

    for day, hours in student_avail.items():
        for hour in hours:
            overlapping_vars = [assignments[var] for var in assignments if var.startswith(f"{student_name}-") and f"-{day}-{hour}" in var]
            if overlapping_vars:
                model.Add(sum(overlapping_vars) <= 1)

# === Prevent overlapping classes per instructor, enforce group class limits ===
for instructor_idx, instructor_row in instructor_df.iterrows():
    instructor_name = instructor_row['Instructor Name']
    instructor_avail = instructor_row['Availability']

    for day, hours in instructor_avail.items():
        for hour in hours:
            overlapping_vars = [(var, var.split('-')[1]) for var in assignments if f"-{instructor_name}-" in var and f"-{day}-{hour}" in var]

            group_vars = [assignments[var] for var, cls in overlapping_vars if cls in group_classes]
            indiv_vars = [assignments[var] for var, cls in overlapping_vars if cls not in group_classes]

            if group_vars:
                model.Add(sum(group_vars) <= GROUP_CLASS_LIMIT)
            if indiv_vars:
                model.Add(sum(indiv_vars) <= 1)

# === No double booking of rooms unless both classes are group classes ===
room_time_assignments = defaultdict(list)

for var_name, var in assignments.items():
    student_name, class_name, instructor_name, day, hour = var_name.rsplit('-', 4)
    class_name_norm = class_name.strip().upper()
    room_var = room_vars[var_name]

    room_time_assignments[(day, hour)].append((var_name, class_name_norm, room_var))

for (day, hour), assignments_at_time in room_time_assignments.items():
    n = len(assignments_at_time)
    for i in range(n):
        var1, cls1, r1 = assignments_at_time[i]
        for j in range(i + 1, n):
            var2, cls2, r2 = assignments_at_time[j]

            if cls1 in group_classes and cls2 in group_classes:
                continue

            b1 = assignments[var1]
            b2 = assignments[var2]

            both_assigned = model.NewBoolVar(f'both_assigned_{var1}_{var2}')
            model.AddBoolAnd([b1, b2]).OnlyEnforceIf(both_assigned)
            model.AddBoolOr([b1.Not(), b2.Not()]).OnlyEnforceIf(both_assigned.Not())

            model.Add(r1 != r2).OnlyEnforceIf(both_assigned)

# === Encourage block schedule consistency ===
def get_block(hour):
    h = int(hour)
    if 9 <= h <= 12:
        return 'morning'
    elif 14 <= h <= 17:
        return 'afternoon'
    return None

for person_df, is_student in [(student_df, True), (instructor_df, False)]:
    for idx, row in person_df.iterrows():
        name = row['Student Name'] if is_student else row['Instructor Name']
        availability = row['Availability']
        modes = row.get('Availability Mode', {})

        for day in availability:
            block_assignments = {'morning': [], 'afternoon': []}
            block_modes = {'morning': set(), 'afternoon': set()}

            for hour in availability[day]:
                block = get_block(hour)
                if not block:
                    continue

                for var in assignments:
                    if f"-{day}-{hour}" in var and name in var:
                        block_assignments[block].append(assignments[var])
                        if not is_student:
                            mode = modes.get(day, '')
                            block_modes[block].add(mode)
                        else:
                            instr_name = var.split('-')[2]
                            instr_row = instructor_df[instructor_df['Instructor Name'] == instr_name]
                            if not instr_row.empty:
                                instr_mode = instr_row.iloc[0]['Availability Mode'].get(day, '')
                                block_modes[block].add(instr_mode)

            for block in ['morning', 'afternoon']:
                if block_assignments[block]:
                    total = len(block_assignments[block])
                    assigned_sum = model.NewIntVar(0, total, f'{name}_{day}_{block}_assigned')
                    model.Add(assigned_sum == sum(block_assignments[block]))

                    is_singleton = model.NewBoolVar(f'{name}_{day}_{block}_singleton')
                    model.Add(assigned_sum == 1).OnlyEnforceIf(is_singleton)
                    model.Add(assigned_sum != 1).OnlyEnforceIf(is_singleton.Not())
                    spread_penalties.append(block_penalty_weight * is_singleton)

                    if len(block_modes[block]) > 1:
                        spread_penalties.append(block_penalty_weight)

# === Objective function ===
model.Maximize(
    sum(assignment_weights[var] * assignments[var] for var in assignments) -
    spread_penalty_weight * sum(spread_penalties)
)

# === Solve ===
solver = cp_model.CpSolver()
status = solver.Solve(model)

# === Output results as in Model A ===
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    # --- Print Student Schedules ---
    for student_idx, student_row in student_df.iterrows():
        student_name = student_row['Student Name']
        student_classes = student_row['Class List']

        print(f"Schedule for {student_name}:")
        student_schedule = {day: [] for day in ['M', 'T', 'W', 'Th', 'F']}
        seen_assignments = set()
        matched_classes = set()
        primary_instrument_hours = 0

        for student_class in student_classes:
            for instructor_idx, instructor_row in instructor_df.iterrows():
                instructor_name = instructor_row['Instructor Name']
                for day in student_row['Availability']:
                    if day in instructor_row['Availability']:
                        common_times = set(student_row['Availability'][day]).intersection(instructor_row['Availability'][day])
                        for time in common_times:
                            var_name = f'{student_name}-{student_class}-{instructor_name}-{day}-{time}'
                            if var_name in assignments and solver.Value(assignments[var_name]) == 1:
                                assigned_room_idx = solver.Value(room_vars[var_name])
                                assigned_room = index_to_room[assigned_room_idx]
                                display_str = f"{student_class} with {instructor_name} at {time} ({instructor_row['Availability Mode'].get(day, '')}), Room: {assigned_room}"
                                if (day, display_str) not in seen_assignments:
                                    student_schedule[day].append(display_str)
                                    seen_assignments.add((day, display_str))
                                    matched_classes.add(student_class)
                                    if student_class == student_row['Primary Instrument']:
                                        primary_instrument_hours += 1

        for day in ['M', 'T', 'W', 'Th', 'F']:
            if student_schedule[day]:
                print(f"  {day}:")
                for entry in sorted(student_schedule[day]):
                    print(f"    {entry}")
        print(f"  ✅ Matched {len(matched_classes)} out of {len(student_classes)} requested classes\n")
        #print(f"Primary instrument hours: {primary_instrument_hours}\n")

    # --- Print Instructor Schedules ---
    for instructor_idx, instructor_row in instructor_df.iterrows():
        instructor_name = instructor_row['Instructor Name']
        instructor_schedule = {day: [] for day in ['M', 'T', 'W', 'Th', 'F']}
        seen_instructor_assignments = set()

        # For each assignment involving this instructor
        for var_name in assignments:
            # Parse var_name format: student-class-instructor-day-time
            parts = var_name.split('-')
            if len(parts) < 5:
                continue
            student_name = parts[0]
            student_class = parts[1]
            instr_name = parts[2]
            day = parts[-2]
            time = parts[-1]

            if instr_name != instructor_name:
                continue
            if solver.Value(assignments[var_name]) == 1:
                assigned_room_idx = solver.Value(room_vars[var_name])
                assigned_room = index_to_room[assigned_room_idx]
                mode = instructor_row['Availability Mode'].get(day, '')
                display_str = f"{student_class} with {student_name} at {time} ({mode}), Room: {assigned_room}"
                if (day, display_str) not in seen_instructor_assignments:
                    instructor_schedule[day].append(display_str)
                    seen_instructor_assignments.add((day, display_str))

        print(f"Schedule for Instructor {instructor_name}:")
        for day in ['M', 'T', 'W', 'Th', 'F']:
            if instructor_schedule[day]:
                print(f"  {day}:")
                for entry in sorted(instructor_schedule[day]):
                    print(f"    {entry}")
        print()
else:
    print("No feasible schedule found.")

'''

Schedule for Aaron :
  T:
    MUSIC THEORY with Siv at 9 (Online), Room: 206B
    PIANO with Siv at 10 (Online), Room: 206B
  F:
    CAREER/JOB COACHING with Em Smith at 10 (In-Person), Room: 101A
    CONCERT REHEARSAL (Saturday Jam) with Rocco Matone at 9 (In-Person), Room: 101A
    GUIDANCE COUNSELING with Em Smith at 12 (In-Person), Room: 101A
  ✅ Matched 5 out of 5 requested classes

Schedule for Brandi Pollard :
  M:
    CHOIR with Evan O’Brien at 16 (Online), Room: 101A
    ENSEMBLE (BAND) with Evan O’Brien at 14 (Online), Room: 206D
  Th:
    CAREER/JOB COACHING with Em Smith at 10 (In-Person), Room: 101A
    CONCERT REHEARSAL (Saturday Jam) with Rocco Matone at 14 (In-Person), Room: 101A
    GUIDANCE COUNSELING with Em Smith at 15 (In-Person), Room: 206D
    MUSIC THEORY with Siv at 12 (Online), Room: 206B
    VOICE with Rocco Matone at 16 (In-Person), Room: 206D
  ✅ Matched 7 out of 7 requested classes

Schedule for El Cid Catlin-Muñoz:
  W:
    CAREER/JOB COACHING with Em Smi